In [2]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
import os
import shutil
import random
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [ ]:
data_path = r"C:\Users\Pc\Desktop\x-ray project  pnumonia\chest x-ray pnumonia\chest_xray"
test_path = r"C:\Users\Pc\Desktop\x-ray project  pnumonia\chest x-ray pnumonia\chest_xray\test"
train_path = r"C:\Users\Pc\Desktop\x-ray project  pnumonia\chest x-ray pnumonia\chest_xray\train"

category = ["NORMAL", "PNEUMONIA"]

# function to clean and  convert images
def clean_and_convert(directory):
    for  category in os.listdir(directory):
        category_path=os.path.join(directory,category)
        for  img_name in os.listdir(category_path):
            img_path=os.path.join(category_path,img_name)
            try:
                with Image.open(img_path) as  img:
                    # convert to RGB  and  save as JPEG if not already
                    if  img.format !="JPEG":
                        img=img.convert("RGB")
                        new_img_path=os.path.splitext(img_path)[0]+'jpg'
                        img.save(new_img_path,'JPEG')
                        os.remove(img_path) # remove the  old file
                        print(f"converted and  saved:{new_img_path}")
            except Exception as e:
                print(f"Removing corrupted or  unreadable image:{img_path} - {e}")
                os.remove(img_path)# remove corrupted or unreadable files

# clean both train and  val directories
clean_and_convert(train_path)
clean_and_convert(test_path)

In [23]:
import shutil

# Define the number of extra copies needed
extra_copies = 2534

# Source and target folders
normal_dir = os.path.join(train_path, "NORMAL")

for i in range(extra_copies):
    src_img = os.path.join(normal_dir, os.listdir(normal_dir)[i % len(os.listdir(normal_dir))])
    dst_img = os.path.join(normal_dir, f"copy_{i}.jpg")
    shutil.copy(src_img, dst_img)

In [3]:
# Image dimensions
image_size = (512, 512)
batch_size = 32

# Image Data Generators (Rescaling & Augmentation)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    zoom_range=0.2
)   

val_datagen = ImageDataGenerator(rescale=1./255)

# Load images
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary"
)


val_generator = val_datagen.flow_from_directory(
    test_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary"
)

# Define CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(512, 512,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification (Pneumonia vs Normal)
])

from tensorflow.keras.callbacks import ReduceLROnPlateau

# Reduce learning rate when the validation loss stops improving
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# Compile model with a smaller initial learning rate
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0001)  # Start small

# Compile model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

# Train model
model.fit(train_generator, validation_data=val_generator, epochs=10, callbacks=[early_stop, lr_scheduler])

# Save trained model
model.save("pneumonia_detection.h5")

print("Model training complete and saved as pneumonia_detection.h5")

Found 7766 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


c:\Users\Pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 1619s 7s/step - accuracy: 0.7783 - loss: 0.6906 - val_accuracy: 0.8397 - val_loss: 0.3653 - learning_rate: 1.0000e-04
Epoch 2/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 1519s 6s/step - accuracy: 0.9102 - loss: 0.2505 - val_accuracy: 0.8446 - val_loss: 0.3469 - learning_rate: 1.0000e-04
Epoch 3/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 1507s 6s/step - accuracy: 0.9222 - loss: 0.2051 - val_accuracy: 0.8478 - val_loss: 0.3625 - learning_rate: 1.0000e-04
Epoch 4/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 1499s 6s/step - accuracy: 0.9235 - loss: 0.2105 - val_accuracy: 0.7933 - val_loss: 0.4485 - learning_rate: 1.0000e-04
Epoch 5/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 1516s 6s/step - accuracy: 0.9260 - loss: 0.2021 - val_accuracy: 0.8670 - val_loss: 0.3195 - learning_rate: 1.0000e-04
Epoch 6/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 1503s 6s/step - accuracy: 0.9292 - loss: 0.1924 - val_accuracy: 0.8750 - val_loss: 0.2994 - learning_rate: 1.0000e-04
Epoch 7/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 1532s 6s/ste

Model training complete and saved as pneumonia_detection.h5


In [4]:
train_generator.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1}

In [16]:
model = tf.keras.models.load_model("pneumonia_detection.h5")

# Load and preprocess test image
img_path = r"C:\Users\Pc\Pictures\pneumon.jfif"
img = image.load_img(img_path, target_size=(512, 512))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict
prediction = model.predict(img_array)
prediction

if prediction > 0.5:
    print("PNEUMONIA")
else:
    print("NORMAL")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
NORMAL


In [5]:
def count_images(directory):
    normal_count = len(os.listdir(os.path.join(directory, "NORMAL")))
    pneumonia_count = len(os.listdir(os.path.join(directory, "PNEUMONIA")))
    print(f"NORMAL: {normal_count} images")
    print(f"PNEUMONIA: {pneumonia_count} images")
    
count_images(train_path)
# count_images(test_path)

NORMAL: 3883 images
PNEUMONIA: 3883 images
